In [45]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier

import sqlite3


In [46]:
# Create connection

con = sqlite3.connect("stellar_class_db")

In [47]:
# Load the data from sqlite
df = pd.read_sql_query("SELECT * FROM stellar", con)
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,mjd,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [48]:
df.set_index(['spec_obj_ID'], inplace = True)

In [49]:
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,mjd,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
6.543777e+18,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,GALAXY,0.634794,5812,56354,171
1.176014e+19,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,GALAXY,0.779136,10445,58158,427
5.152200e+18,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,GALAXY,0.644195,4576,55592,299
1.030107e+19,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,GALAXY,0.932346,9149,58039,775
6.891865e+18,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.055431e+19,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,GALAXY,0.000000,9374,57749,438
8.586351e+18,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,GALAXY,0.404895,7626,56934,866
3.112008e+18,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,GALAXY,0.143366,2764,54535,74


In [50]:
 df.drop(['run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'mjd', 'fiber_ID'], axis = 1 )

,alpha,delta,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,,,
6.543777e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1.176014e+19,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
5.152200e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
1.030107e+19,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
6.891865e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
...,...,...,...,...,...,...,...,...,...
1.055431e+19,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000
8.586351e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895
3.112008e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366


In [51]:
df.to_csv("machine_learning_model.csv")

In [52]:
df["z"].min()

-9999.0

In [53]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
df["binned_u"] = pd.cut(df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
df["binned_g"] = pd.cut(df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
df["binned_r"] = pd.cut(df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
df["binned_i"] = pd.cut(df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
df["binned_z"] = pd.cut(df["z"], z_size_bins, labels = frequency)

df.sample(5)

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,...,class,redshift,plate,mjd,fiber_ID,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,,,,,,,,
1.057349e+19,1.237666e+18,18.217083,-0.540431,22.85040,22.86227,22.41734,21.71473,20.78993,4858,301,...,GALAXY,0.894806,9391,58072,604,3,3,2,2,3
7.067437e+18,1.237680e+18,3.646709,29.245675,22.01536,21.08234,19.93061,19.22048,18.62015,8116,301,...,GALAXY,0.527434,6277,56273,595,3,3,2,2,3
4.205459e+18,1.237666e+18,16.449638,0.388681,23.34055,22.25369,21.75873,21.73260,23.14428,4858,301,...,STAR,-0.000388,3735,55209,810,3,3,2,2,3
1.030989e+19,1.237664e+18,355.571575,-0.745703,22.76569,22.63338,21.80171,21.57209,21.50589,4263,301,...,GALAXY,0.478479,9157,57742,94,3,3,2,2,3
6.954709e+18,1.237679e+18,1.516206,13.611375,21.95325,20.94780,18.96115,18.27121,17.77286,7773,301,...,GALAXY,0.364076,6177,56268,92,3,3,2,2,3


In [54]:
# Create our features
X = df.drop('class', axis=1)

# Create our target
y = df['class']

In [55]:
df.to_csv("mlm_results.csv")

In [14]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'STAR': 44608, 'GALAXY': 44608, 'QSO': 44608})

In [16]:
# Train the Logistic Regression Model using the resampled data
model = RandomForestClassifier(random_state=1)
model.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9745176569907222

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.98      0.98      0.97      0.98      0.98      0.95     14837
        QSO       0.95      0.94      0.99      0.95      0.97      0.93      4796
       STAR       0.98      1.00      1.00      0.99      1.00      1.00      5367

avg / total       0.98      0.98      0.98      0.98      0.98      0.96     25000



In [19]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cm = pd.DataFrame(c, index=["Actual 0", "Actual 1", "Actual 2"], columns=["Predicted 0", "Predicted 1", "Predicted 2"])
cm

,Predicted 0,Predicted 1,Predicted 2
Actual 0,14525,216,96
Actual 1,264,4532,0
Actual 2,2,0,5365


In [56]:
# Write new df to new SQLite table - 

df.to_sql("mlm_results", con, if_exists="replace")

In [57]:
# Close connection
con.close()

In [58]:
import pandas as pd
from config import password
from sqlalchemy import create_engine
engine = create_engine("postgresql://postgres:" + password + "@localhost:5432/Stellar_Classification")

In [59]:
df = pd.read_csv("mlm_results.csv")
df.to_sql("mlm_results", engine, if_exists="replace")